In [39]:
import numpy as np
import pandas as pd
import nannyml as nml
from sklearn.datasets import fetch_california_housing

In [6]:
# load original data
data, target = fetch_california_housing(as_frame=True, return_X_y=True)
timestamp = pd.date_range(start='1/1/2018', periods=len(data), freq='H')
data.index = timestamp
target.index = timestamp

# load aging results
aging_results = pd.read_parquet('../results/aging/cal_house/aging_cal_house_1_simulations_3000_prod.parquet')

In [7]:
target

2018-01-01 00:00:00    4.526
2018-01-01 01:00:00    3.585
2018-01-01 02:00:00    3.521
2018-01-01 03:00:00    3.413
2018-01-01 04:00:00    3.422
                       ...  
2020-05-09 19:00:00    0.781
2020-05-09 20:00:00    0.771
2020-05-09 21:00:00    0.923
2020-05-09 22:00:00    0.847
2020-05-09 23:00:00    0.894
Freq: H, Name: MedHouseVal, Length: 20640, dtype: float64

In [8]:
aging_results

,y,y_pred,partition,timestamp,model_age,is_model_valid,simulation_id
2018-03-31 10:00:00,0.647,0.659553,train,2018-03-31 10:00:00,-313,True,0
2018-03-31 11:00:00,0.673,0.777707,train,2018-03-31 11:00:00,-313,True,0
2018-03-31 12:00:00,0.677,0.895240,train,2018-03-31 12:00:00,-313,True,0
2018-03-31 13:00:00,0.619,0.714690,train,2018-03-31 13:00:00,-313,True,0
2018-03-31 14:00:00,0.581,0.653271,train,2018-03-31 14:00:00,-313,True,0
...,...,...,...,...,...,...,...
2020-04-26 23:00:00,3.912,3.409645,prod,2020-04-26 23:00:00,445,True,0
2020-04-27 00:00:00,4.521,2.945761,prod,2020-04-27 00:00:00,445,True,0
2020-04-27 01:00:00,2.287,1.877643,prod,2020-04-27 01:00:00,445,True,0
2020-04-27 02:00:00,3.406,2.652814,prod,2020-04-27 02:00:00,445,True,0


In [21]:
simulation_ids = aging_results['simulation_id'].unique()
nml_data = aging_results.merge(data, left_index=True, right_index=True, how='left')

for simulation_id in simulation_ids:
    print(simulation_id)
    simulation_df = nml_data[nml_data['simulation_id'] == simulation_id]
    
    # get original test set
    test_aging_results = simulation_df[simulation_df['partition'] == 'test']
    idx_test_start, idx_test_end = test_aging_results.index[0], test_aging_results.index[-1]
    reference_df = nml_data[idx_test_start:idx_test_end]
    
    # get original prod set
    prod_aging_results = simulation_df[simulation_df['partition'] == 'prod']
    idx_prod_start, idx_prod_end = prod_aging_results.index[0], prod_aging_results.index[-1]
    analysis_df = nml_data[idx_prod_start:idx_prod_end]
    
    # fit DLE from NannyML
    estimator = nml.DLE(
        feature_column_names=data.columns.tolist(),
        y_pred='y_pred',
        y_true='y',
        timestamp_column_name='timestamp',
        metrics=['mape'],
        chunk_period='d',
        tune_hyperparameters=False
    )
    
    estimator.fit(reference_df)
    # performance estimation results
    pe_results = estimator.estimate(analysis_df)
    
    calculator = nml.PerformanceCalculator(
        y_pred='y_pred',
        y_true='y',
        timestamp_column_name='timestamp',
        metrics=['mape'],
        chunk_period='d',
        problem_type='regression'
    ).fit(reference_df)
    realized_results = calculator.calculate(analysis_df)

0


/Users/santiago/miniconda3/envs/thesis/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning:

Using categorical_feature in Dataset.



In [22]:
pe_results.filter(period='analysis').plot()

In [25]:
pe_results.filter(period='analysis').compare(realized_results).plot().show()

In [29]:
pe_results.filter(period='analysis').compare(realized_results)

In [42]:
realized_results.filter(period='analysis').to_df()['mape']['alert'].value_counts()

True     84
False    42
Name: alert, dtype: int64

In [40]:
pe_comparison = pe_results.filter(period='analysis').to_df()['mape']
pe_comparison['estimated_alert'] = pe_comparison['alert']
pe_comparison['realized_alert'] = np.where(pe_comparison['realized'] > pe_comparison['upper_threshold'], 
                                           True, False)

In [50]:
true_alerts = pe_comparison[pe_comparison['realized_alert'] == True]

sum(true_alerts['estimated_alert'] == true_alerts['realized_alert'])

3

In [51]:
true_alerts

,sampling_error,realized,value,upper_confidence_boundary,lower_confidence_boundary,upper_threshold,lower_threshold,alert,estimated_alert,realized_alert
0,0.034603,0.337411,0.149554,0.253363,0.045745,0.229802,0.0,False,False,True
1,0.031588,0.344816,0.135867,0.230630,0.041103,0.229802,0.0,False,False,True
2,0.031588,0.269388,0.131699,0.226463,0.036935,0.229802,0.0,False,False,True
3,0.031588,0.289675,0.136579,0.231343,0.041815,0.229802,0.0,False,False,True
5,0.031588,0.302518,0.119472,0.214236,0.024709,0.229802,0.0,False,False,True
...,...,...,...,...,...,...,...,...,...,...
110,0.031588,0.236067,0.163431,0.258195,0.068667,0.229802,0.0,False,False,True
111,0.031588,0.255643,0.147629,0.242392,0.052865,0.229802,0.0,False,False,True
116,0.031588,0.242066,0.134830,0.229594,0.040066,0.229802,0.0,False,False,True
124,0.031588,0.296796,0.142500,0.237264,0.047736,0.229802,0.0,False,False,True


fit nannyml with more than the test set. Fit it with the test + data from test to before prod.